In [13]:
import pandas as pd
import mysql.connector
import getpass,os

from mysql_connect import MySqlConnection

import numpy as np
import pandas as pd
import os
from pathlib import Path


In [14]:
### Mysql Server Connection 

budget_schema_config = {
    "host":"127.0.0.1",
    "user":"analysis"

}

mysql_conn = MySqlConnection(**budget_schema_config)
mysql_conn.connect()

MySql Server Connection Established...

Connection Details:
User --> analysis@localhost
Date Time --> 2025-03-18 10:55:21 PM


In [15]:
db_name='SQL_PRACTICE'
mysql_conn.create_datebase(db_name=db_name)

Database SQL_PRACTICE created successfully


#### Weather

In [16]:
table_name = "Weather"

## Create Table Script
create_weather = f"""drop table if exists {table_name}; 
                        Create table If Not Exists {table_name} 
                        (id int, recordDate date, temperature int)
                        """

## Data Base Conection 
mysql_conn.select_database(db_name=db_name)

## Create Table
mysql_conn.create_table(create_table_query=create_weather,table_name=table_name)

insert_script = """ insert into Weather (id, recordDate, temperature) values ('1', '2015-01-01', '10')
                    insert into Weather (id, recordDate, temperature) values ('2', '2015-01-02', '25')
                    insert into Weather (id, recordDate, temperature) values ('3', '2015-01-03', '20')
                    insert into Weather (id, recordDate, temperature) values ('4', '2015-01-04', '30') """

mysql_conn.select_database(db_name=db_name)
for row in insert_script.split('\n'):
    insert_script = row
    mysql_conn.insert_record(insert_script=insert_script, table_name=table_name)

Connected to database: SQL_PRACTICE
Table [Weather] created successfully.
Reconnecting to MySQL...
MySql Server Connection Established...

Connection Details:
User --> analysis@localhost
Date Time --> 2025-03-18 10:55:21 PM
Connected to database: SQL_PRACTICE
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.


In [17]:
mysql_conn.select_database(db_name=db_name)


Connected to database: SQL_PRACTICE


#### Write a solution to find all dates' id with higher temperatures compared to its previous dates (yesterday).

In [19]:
data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})
weather

,id,recordDate,temperature
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


In [20]:

sql = """
SELECT w1.id
FROM Weather w1, Weather w2
WHERE DATEDIFF(w1.recordDate, w2.recordDate) = 1 AND w1.temperature > w2.temperature;
""" 
mysql_conn.select_table_query(sql)

,id
0,2
1,4


MySql Connection Closed.........2025-03-18 10:40:57 PM



,id,recordDate,temperature
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


In [21]:
table_name = "Activity"

## Create Table Script
create_weather = f"""drop table if exists {table_name}; 
                    Create table If Not Exists {table_name}  
                    (machine_id int, process_id int, activity_type ENUM('start', 'end'), timestamp float)
                    """

## Data Base Conection 
mysql_conn.select_database(db_name=db_name)

## Create Table
mysql_conn.create_table(create_table_query=create_weather,table_name=table_name)

insert_script = """ insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '0', 'start', '0.712')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '0', 'end', '1.52')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '1', 'start', '3.14')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('0', '1', 'end', '4.12')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '0', 'start', '0.55')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '0', 'end', '1.55')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '1', 'start', '0.43')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('1', '1', 'end', '1.42')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '0', 'start', '4.1')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '0', 'end', '4.512')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '1', 'start', '2.5')
insert into Activity (machine_id, process_id, activity_type, timestamp) values ('2', '1', 'end', '5') """

mysql_conn.select_database(db_name=db_name)
for row in insert_script.split('\n'):
    insert_script = row
    mysql_conn.insert_record(insert_script=insert_script, table_name=table_name)

Connected to database: SQL_PRACTICE
Table [Activity] created successfully.
Reconnecting to MySQL...
MySql Server Connection Established...

Connection Details:
User --> analysis@localhost
Date Time --> 2025-03-18 11:05:52 PM
Connected to database: SQL_PRACTICE
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.
rows inserted successfully.


##### The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the machine_id along with the average time as processing_time, which should be rounded to 3 decimal places.

Return the result table in any order.

In [22]:

sql = """
SELECT MACHINE_ID , ROUND(avg(TOTAL_TIMESTAMP),3) AS PROCESSING_TIME FROM (
select A1.*, 
A2.TIMESTAMP - A1.TIMESTAMP AS Total_timestamp
from ACTIVITY A1
JOIN ACTIVITY A2
ON A1.MACHINE_ID = A2.MACHINE_ID AND A1.PROCESS_ID = A2.PROCESS_ID
WHERE A1.ACTIVITY_TYPE = 'start' and A2.ACTIVITY_TYPE = 'end'
) AS FINAL
GROUP BY MACHINE_ID;
""" 
mysql_conn.select_table_query(sql)

,machine_id,PROCESSING_TIME
0,0,0.894
1,1,0.995
2,2,1.456


In [23]:
mysql_conn.connection_close()

MySql Connection Closed.........2025-03-19 12:01:00 AM

